# Shakespearean text

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
shakespear_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespear_url)

with open(filepath) as f:
    shakespear_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [23]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespear_text])
dataset_size = tokenizer.document_count
max_id = len(tokenizer.word_index)

In [24]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespear_text])) - 1

In [25]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [26]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [27]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [28]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [29]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [30]:
dataset = dataset.prefetch(1)

In [33]:
 model = keras.models.Sequential([
 keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
 dropout=0.2, recurrent_dropout=0.2),
 keras.layers.GRU(128, return_sequences=True,
 dropout=0.2, recurrent_dropout=0.2),
 keras.layers.TimeDistributed(keras.layers.Dense(max_id,
 activation="softmax"))
 ])
 model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", run_eagerly=True)
 history = model.fit(dataset, epochs=20)

Epoch 1/20


ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.